# Prima prova di neural network

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
#from tqdm.notebook import tqdm, trange
from tqdm.auto import tqdm, trange
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.io import imread
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pickle
import json
import sys
import time
from importlib import reload

In [ ]:
COLOR = True

prendo i dati, li leggo e li splitto

In [ ]:
with open('../data/images/data.json', 'r') as f:
    train = json.load(f)
labels = {}
train_img = []
y = []
for img_name, type in tqdm(train.items()):
    labels[type] = True
    y.append(type)
    image_path = '../data/' + str(img_name)
    img = imread(image_path, as_gray=not COLOR)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    train_img.append(img)


numLabel = len(labels)
# converting the list to numpy array
X = np.array(train_img)
y = np.array(y)

One hot encoding o Label encoding

In [ ]:
oneHot = False

In [ ]:
if oneHot:
    one = preprocessing.OneHotEncoder()
    y = one.fit_transform(y.reshape(-1,1)).toarray()
    pickle.dump(one, open(f'saves/oneHotEncoder{"Color" if COLOR else ""}.pickle', 'wb'))
else:
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    # print the classes
    print(le.classes_)
    pickle.dump(le, open(f'saves/labelEncoder{"Color" if COLOR else ""}.pickle', 'wb'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.15)


# converting the target into torch format
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)
X_train  = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

# shape of training data
X_train.shape, y_train.shape

In [ ]:
if COLOR:
    # inverto gli assi, l'rgb non e' sui singoli pixel ma diviso in tre canali diversi di immagini con pixel monovalore
    X_test = X_test.permute(0, 3, 2, 1)
    X_train = X_train.permute(0, 3, 2, 1)
else:
    # aggiungo la dimensione canale(colore), che qui e' solo una
    X_train = X_train.unsqueeze(1)
    X_test = X_test.unsqueeze(1)
y_test = y_test.type(torch.LongTensor)
y_train = y_train.type(torch.LongTensor)

In [ ]:
test = TensorDataset(X_test, y_test)
train = TensorDataset(X_train, y_train)

creo i batch, randomizzando i dati, con un dataloader

In [ ]:
batch_size = 50

# Create data loaders.
test = DataLoader(test, batch_size=batch_size, shuffle=True)
train = DataLoader(train, batch_size=batch_size, shuffle=True)

for X, y in test:
    print(f"Shape of X : {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# creo la neural network

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# moved definition to a second file, so I can dump the model and reload it from a third file
import CNN as cnn
reload(cnn)

In [ ]:
model = cnn.NeuralNetwork(numLabel=numLabel, not_color= not COLOR).to(device)
print(model)

In [ ]:
#loss_fn = nn.MSELoss()
#loss_fn = nn.L1Loss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-7, momentum=0.9)
LEARNING_RATE = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()

In [ ]:
def trainf(dataloader, model, loss_fn, optimizer):
    global best_model
    best_loss = float("inf")
    size = len(dataloader.dataset)
    #model.double()
    model.train()
    for batch , (X, Y) in tqdm(enumerate(dataloader), leave=False, total=len(dataloader), position=1, file=sys.stdout):
        X, Y = X.to(device), Y.to(device)
        loss = float("inf")
        optimizer.zero_grad()
        # Compute prediction error
        #output = torch.argmax(X, dim=1).float()
        pred = model(X)
        loss = loss_fn(pred,Y)
        loss.backward()
        # Backpropagation
        optimizer.step()

        if loss < best_loss:
            best_loss = loss
            best_model = deepcopy(model)

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            train_losses.append(loss)
            tqdm.write(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def testf(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            gas_loss = [([],[]) for _ in range(5)]
            X, Y = X.to(device), Y.to(device)
            pred = model(X)
            loss = loss_fn(pred, Y).item()
            test_loss += loss
            if batch % 100 == 0:
                val_losses.append(loss)
    test_loss /= num_batches * batch_size
    tqdm.write(f"Test Error:  Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 20
val_losses = []
train_losses = []
best_model = None
for t in trange(epochs, file=sys.stdout):
    trainf(train, model, loss_fn, optimizer)
    testf(test, model, loss_fn)
torch.save(model.state_dict(), f'saves/nn{"Color" if COLOR else ""}.torch')

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()